# Comment finetuner des LLMs avec des adaptateurs LoRA en utilisant Hugging Face TRL

Ce *notebook* montre comment finetuner efficacement de grands modèles de langage en utilisant des adaptateurs LoRA (*Low-Rank Adaptation*). LoRA est une technique de finetuning efficace en termes de paramètres qui :
- gèle les poids du modèle pré-entraîné
- ajoute aux couches d'attention de petites matrices de décomposition de rangs entraînables
- Réduit généralement les paramètres entraînables d'environ 90%
- Maintient les performances du modèle tout en étant économe en mémoire

Nous aborderons les points suivants
1. Mise en place de l'environnement de développement et configuration de LoRA
2. Créer et préparer le jeu de données pour l'entraînement de l'adaptateur
3. Finetuner en utilisant `trl` et `SFTTrainer` avec les adaptateurs LoRA
4. Tester le modèle et fusionner les adaptateurs (optionnel)

## 1. Configurer l'environnement de développement

Notre première étape consiste à installer les bibliothques d'Hugging Face et Pytorch, y compris trl, les transformers et les datasets. Si vous n'avez pas encore entendu parler de trl, ne vous inquiétez pas. Il s'agit d'une nouvelle bibliothèque au-dessus des transformers et des datasets permetant de finetuner, rlhf, aligner les LLMs ouverts plus facilement.

In [ ]:
# Installer les prérequis dans Google Colab
# !pip install transformers datasets trl huggingface_hub

# S'authentifier sur Hugging Face
from huggingface_hub import login

login()

# Pour plus de facilité, vous pouvez créer une variable d'environnement contenant votre jeton de hub sous la forme HF_TOKEN

## 2. Charger le jeu de données

In [13]:
# Charger un échantillon de jeu de données
from datasets import load_dataset

# TODO : définir votre jeu de données et votre configuration en utilisant les paramètres path et name
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. Fine-tune LLM using `trl` and the `SFTTrainer` with LoRA

The [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) from `trl` provides integration with LoRA adapters through the [PEFT](https://huggingface.co/docs/peft/en/index) library. Key advantages of this setup include:

1. **Memory Efficiency**: 
   - Only adapter parameters are stored in GPU memory
   - Base model weights remain frozen and can be loaded in lower precision
   - Enables fine-tuning of large models on consumer GPUs

2. **Training Features**:
   - Native PEFT/LoRA integration with minimal setup
   - Support for QLoRA (Quantized LoRA) for even better memory efficiency

3. **Adapter Management**:
   - Adapter weight saving during checkpoints
   - Features to merge adapters back into base model

We'll use LoRA in our example, which combines LoRA with 4-bit quantization to further reduce memory usage without sacrificing performance. The setup requires just a few configuration steps:
1. Define the LoRA configuration (rank, alpha, dropout)
2. Create the SFTTrainer with PEFT config
3. Train and save the adapter weights


In [ ]:
# Importer les bibliothèques nécessaires
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Charger le modèle et le tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Définir le format de chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Définir le nom du finetuning à sauvegarder et/ou à télécharger
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

`SFTTrainer` supporte une intégration native avec `peft`, ce qui rend super facile le finetuning des LLMs en utilisant, par exemple, LoRA. Nous avons seulement besoin de créer notre `LoraConfig` et de le fournir au Trainer.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercice : Définir les paramètres de LoRA pour le finetuning</h2>
    <p>Prenez un jeu de données provenant du Hub d'Hugging Face et finetuné un modèle sur dessus. </p> 
    <p><b>Niveaux de difficulté</b></p>
    <p>🐢 Utiliser les paramètres généraux pour un finetuning arbitraire</p>
    <p>🐕 Ajuster les paramètres et vérifier les poids et les biais</p>
    <p>🦁 Ajuster les paramètres et montrer les changements dans les résultats de l'inférence</p>
</div>

In [ ]:
from peft import LoraConfig

# TODO : Configurer les paramètres de LoRA
# r : dimension du rang des matrices LoRA (plus petite = plus de compression)
rank_dimension = 6
# lora_alpha : facteur d'échelle pour les couches LoRA (plus élevé = adaptation plus forte)
lora_alpha = 8
# lora_dropout : probabilité de dropout pour les couches LoRA (aide à prévenir le surentraînement)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Dimension du rang, généralement entre 4 et 32
    lora_alpha=lora_alpha,  # Facteur d'échelle LoRA, généralement 2x le rang
    lora_dropout=lora_dropout,  # Probabilité de dropout probability pour les couches de LoRA
    bias="none",  # Type de biais pour le LoRA. Les biais correspondants seront mis à jour pendant l'entraînement
    target_modules="all-linear",  # Modules auxquels appliquer le LoRA
    task_type="CAUSAL_LM",  # Type de tâche pour l'architecture du modèle
)

Avant de commencer notre entraînement, nous devons définir les hyperparamètres (`TrainingArguments`) que nous voulons utiliser.

In [ ]:
# Configuration de l'entraînement
# Hyperparamètres basés sur les recommandations du papier du QLoRA
args = SFTConfig(
    # Paramètres de sortie
    output_dir=finetune_name,  # Répertoire pour enregistrer les checkpoints du modèle
    # Durée de l'entraînement
    num_train_epochs=1,  # Nombre d'époques d'entraînement
    # Paramètres de la taille des batchs
    per_device_train_batch_size=2,  # Taille des batchs par GPU
    gradient_accumulation_steps=2,  # Accumuler les gradients pour obtenir un plus grand batch efficace
    # Optimisation de la mémoire
    gradient_checkpointing=True,  # Échanger le calcul contre des économies de mémoire
    # Paramètres de l'optimiseur
    optim="adamw_torch_fused",  # Utiliser AdamW fusionné pour plus d'efficacité
    learning_rate=2e-4,  # Taux d'apprentissage (papier du QLoRA)
    max_grad_norm=0.3,  # Seuil d'écrêtage du gradient
    # Taux d'apprentissage
    warmup_ratio=0.03,  # Portion de pas pour l'échauffement
    lr_scheduler_type="constant",  # Maintenir un rythme d'apprentissage constant après l'échauffement
    # Enregistrement et sauvegarde
    logging_steps=10,  # Enregistrement des métriques tous les N pas
    save_strategy="epoch",  # Sauvegarde du checkpoint à chaque époque
    # Paramètres de précision
    bf16=True,  # Utiliser la précision bfloat16
    # Paramètres d'intégration
    push_to_hub=False,  # Ne pas pousser vers le Hub
    report_to="none",  # Désactiver l'enregistrement externe
)

Nous avons maintenant tous les éléments nécessaires pour créer notre `SFTTrainer` et commencer à entraîner notre modèle.

In [ ]:
max_seq_length = 1512  # longueur maximale des séquences pour le modèle et le paquetage du jeu de données

# Créer SFTTrainer avec la configuration LoRA
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # Configuration LoRA
    max_seq_length=max_seq_length,  # Longueur maximale de la séquence
    tokenizer=tokenizer,
    packing=True,  # Activer l'emballage d'entrée pour plus d'efficacité
    dataset_kwargs={
        "add_special_tokens": False,  # Tokens spéciaux gérés par le gabarit
        "append_concat_token": False,  # Aucun séparateur supplémentaire n'est nécessaire
    },
)

Commencez à entraîner notre modèle en appelant la méthode `train()` sur notre instance `Trainer`. Cela va démarrer la boucle d'entraînement et entraîner notre modèle pendant 3 époques. Puisque nous utilisons une méthode PEFT, nous ne sauvegarderons que les poids du modèle adapté et non le modèle complet.

In [ ]:
# commencer l'entraînement, le modèle sera automatiquement sauvegardé sur le Hub et dans le répertoire de sortie.
trainer.train()

# Sauvegarder le modèle
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

L'entraînement avec Flash Attention pour 3 époques avec un jeu de données de 15k échantillons a pris 4:14:36 sur un `g5.2xlarge`. L'instance coûte `1.21$/h` ce qui nous amène à un coût total de seulement ~`5.3$`.



### Fusionner l'adaptateur LoRA dans le modèle original

Lors de l'utilisation de LoRA, nous n'entraînons que les poids de l'adaptateur tout en gardant le modèle de base gelé. Pendant l'entraînement, nous sauvegardons uniquement ces poids d'adaptateur légers (~2-10MB) plutôt qu'une copie complète du modèle. Cependant, pour le déploiement, vous pouvez vouloir fusionner les adaptateurs dans le modèle de base pour :

1. **Déploiement simplifié** : Fichier de modèle unique au lieu du modèle de base + adaptateurs
2. **Vitesse d'inférence** : Pas de surcharge de calcul des adaptateurs
3. **Compatibilité avec les frameworks** : Meilleure compatibilité avec les frameworks

In [ ]:
from peft import AutoPeftModelForCausalLM


# Chargement du modèle PEFT sur le CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Fusionner le modèle LoRA et le modèle de base et sauvegarder
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. Tester le modèle et exécuter l'inférence

Une fois l'entraînement terminé, nous voulons tester notre modèle. Nous allons charger différents échantillons du jeu de données original et évaluer le modèle sur ces échantillons, en utilisant une boucle simple et l'*accuracy* comme métrique.


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercice bonus : Chargement de l'adaptateur LoRA</h2>
    <p>Utilisez ce que vous avez appris dans le notebook pour charger votre adaptateur LoRA entraîné pour l'inférence</p> 
</div>

In [30]:
# libérer la mémoire
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Chargement du modèle avec l'adaptateur PEFT
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

Testons quelques échantillons d'instructions et voyons comment le modèle se comporte.

In [34]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)